In [ ]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define image directories
train_dir = '/kaggle/input/emirhan-human-dataset/human_data/train_data'
test_dir = '/kaggle/input/emirhan-human-dataset/human_data/test_data'

# Define ImageDataGenerators for training and testing datasets
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Rescale pixel values to [0, 1]
    shear_range=0.2,
    horizontal_flip=True,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Define batch size and image size
batch_size = 32
image_size = (224, 224)  # Target image size

# Flow images from directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = image_size,
    batch_size=batch_size,
    class_mode='categorical'  # for multi-class classification
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = image_size,
    batch_size=batch_size,
    class_mode='categorical'  # for multi-class classification
)

In [ ]:
import os
import matplotlib.pyplot as plt

# Get a batch of data from the generator
images, labels = next(train_generator)

print(labels.shape)
# Get the class names from the directory names
class_names = list(train_generator.class_indices.keys())
print(train_generator.class_indices)
# Visualize a sample image and its inferred label
sample_index = 0  # Index of the sample image to visualize
for i in range(0, 5):
    plt.imshow(images[i])
    plt.title("Label: " + class_names[labels[i].argmax()])
    plt.axis("off")
    plt.show()

In [ ]:
from tensorflow.keras.applications import DenseNet169
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

base_model = DenseNet169(weights='imagenet', include_top=False)
num_classes = len(class_names)
# Add custom top layers for your specific task
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)  # Modify num_classes as per your task

# Combine the base model with custom top layers
model = Model(inputs=base_model.input, outputs=predictions)

# UnFreeze the layers of the base model
for layer in base_model.layers:
    layer.trainable = True

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Print model summary
model.summary()

In [ ]:
# Train the model with distributed data
from tensorflow.keras.callbacks import EarlyStopping

# Define early stopping callback
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Fit the model with early stopping
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=test_generator,
    callbacks=[early_stopping]
)

In [ ]:
# Plot training and validation loss
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
# Plot training and validation accuracy
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Training and Validation Accuracy')
plt.legend()
plt.show()

In [ ]:
# Save the model weights
model.save('/kaggle/working/my_model.keras')

In [ ]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('/kaggle/working/my_model.keras')  # Replace 'your_model.h5' with the path to your saved model file

# Create an ImageDataGenerator for the test dataset
test_datagen = ImageDataGenerator(rescale=1.0/255)  # You may need to adjust other parameters based on how you preprocess your images
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),  # Adjust based on your input image size
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Make predictions on the test data
predictions = model.predict(test_generator)

# Get true labels
true_labels = test_generator.classes

# Get predicted labels
predicted_labels = np.argmax(predictions, axis=1)

# Compute confusion matrix
conf_matrix = confusion_matrix(true_labels, predicted_labels)

# Plot confusion matrix
class_names = test_generator.class_indices.keys()
disp = ConfusionMatrixDisplay(confusion_matrix=conf_matrix, display_labels=class_names)
plt.figure(figsize=(10, 8))
disp.plot(cmap=plt.cm.Blues, xticks_rotation='vertical')
plt.title('Confusion Matrix')
plt.show()